In [36]:
from transformers import BertTokenizer
# load tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [35]:
sentence = '@philly European Central Bank goes bullish on bitcoin: â€œ[â€¦] recent developments, such as the listing of Bitcoin futures contracts by US exchanges, could lead European banks too to hold positions in Bitcoin, and therefore we will certainly look at that,â€ - ECB President Mario Draghi  https://t.co/lDgJU1H2X1'
print('before preprocessing: \n\n'+sentence)

# apply the following preprocessing techniques on the data 
import re 
# lowercase all characters 
sentence = sentence.lower()

# remove URL's 
sentence = re.sub(r"http\S+", '', sentence)
sentence = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', sentence)

# remove HTML reference characters 
sentence = re.sub(r"&[a-z]+;", '', sentence)

# remove non english letters 
sentence = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', sentence)
sentence 

before preprocessing: 

@philly European Central Bank goes bullish on bitcoin: â€œ[â€¦] recent developments, such as the listing of Bitcoin futures contracts by US exchanges, could lead European banks too to hold positions in Bitcoin, and therefore we will certainly look at that,â€ - ECB President Mario Draghi  https://t.co/lDgJU1H2X1


'philly european central bank goes bullish on bitcoin: ] recent developments such as the listing of bitcoin futures contracts by us exchanges could lead european banks too to hold positions in bitcoin and therefore we will certainly look at that - ecb president mario draghi  '

In [ ]:
# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(test_sentence))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentence)))